```
bash 1_extract_memory_info.sh /oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test/7_pfactor_v5/INI50/2_refit/snpnet.log > INI50_pfactor_v5_refit.log
```

In [1]:
import numpy as np
import pandas as pd

In [36]:
log='INI50_pfactor_v5_refit.log'
out_tsv='INI50_pfactor_v5_refit.mem.tsv'

In [3]:
with open(log) as f:
    lines=f.read().splitlines()

In [32]:
def get_numbers(lines, keyword, colidx=-1):
    iteration_idx = np.where([keyword in lines[x] for x in range(len(lines))])[0]    
    return np.array([
        iteration_idx,        
        np.array([float(lines[x].split()[colidx]) for x in iteration_idx])
    ]).transpose()

def get_iteration_idx(lines):
    return np.array(get_numbers(lines, 'Iteration'), dtype=np.int)

def get_num_variables(lines):
    return np.array(get_numbers([x.replace('.', '') for x in lines], 'strong set'), dtype=np.int)

def get_Nmem(lines):
    return get_numbers(lines, 'Ncells', colidx=2)

def get_Vmem(lines):
    return get_numbers(lines, 'Vcells', colidx=2)

def filter_df(df):
    return df[[True] + list(np.array(list(df['lambda_idx'][1:])) - np.array(list(df['lambda_idx'][:-1])) == 1)]    

In [22]:
iteration_idx = get_iteration_idx(lines)


In [23]:
iteration_idx_d = dict(iteration_idx)
for i in range(len(lines)):
    if not i in iteration_idx_d:
        iteration_idx_d[i] = iteration_idx_d[i-1]

In [24]:
nv_df = pd.DataFrame(get_num_variables(lines), columns=['line', 'num_variables'])
nmem_df = pd.DataFrame(get_Nmem(lines), columns=['line', 'Ncells'])
vmem_df = pd.DataFrame(get_Vmem(lines), columns=['line', 'Vcells'])


In [25]:
for df in [nv_df, nmem_df, vmem_df]:
    df['lambda_idx'] = df['line'].map(lambda x: iteration_idx_d[x])
    

In [33]:
merged_df = filter_df(nv_df)[['lambda_idx', 'num_variables']].merge(
    filter_df(nmem_df)[['lambda_idx', 'Ncells']]
).merge(
    filter_df(vmem_df)[['lambda_idx', 'Vcells']]
)
merged_df['mem'] = merged_df['Ncells'] + merged_df['Vcells']


In [37]:
merged_df.to_csv(out_tsv, sep='\t', index=False)

In [38]:
merged_df

,lambda_idx,num_variables,Ncells,Vcells,mem
0,1,1012,210.4,2460.3,2670.7
1,2,1017,210.9,2487.1,2698.0
2,3,1030,210.9,2531.9,2742.8
3,4,1064,211.0,2619.9,2830.9
4,5,1126,211.0,2765.4,2976.4
...,...,...,...,...,...
88,89,115333,216.9,237767.3,237984.2
89,90,120333,217.1,248040.7,248257.8
90,91,125833,217.4,259348.3,259565.7
91,92,126333,217.5,260399.3,260616.8
